In [1]:
from PyQt5 import QtCore, QtGui, QtWidgets
import pathlib

In [2]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.InteractiveSession(config=config)
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

C:\Users\umang\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\umang\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\umang\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\umang\Anaconda3\envs\tensorflow\lib\site-packages

In [3]:
from xml.etree.ElementTree import ElementTree
from xml.etree.ElementTree import Element
import xml.etree.ElementTree as etree

In [4]:
import sys
sys.path.append(r'C:\Users\umang\Anaconda3\envs\tensorflow\intern_project\models\research')
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [5]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

In [6]:
from object_detection import get_inference

Using TensorFlow backend.


In [7]:
PATH_TO_TEST_IMAGES_DIR = pathlib.Path("models/research/object_detection/test_images")
PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
image_save_path = 'models/research/object_detection/annoted_images/mod_image'
i = 0
threshold = 0.7
MODEL_NAME = 0
FILTER = 0

In [8]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [9]:
def load_image_into_numpy_array(image): 
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

In [10]:
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('models/research/object_detection/test_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS
IMAGE_SIZE = (12,8)

In [11]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.compat.v1.Session() as sess:
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
        'num_detections', 'detection_boxes', 'detection_scores',
        'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(tensor_name)
      if 'detection_masks' in tensor_dict:
        #The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        real_num_detections = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1]) 
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1]) 
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[1], image.shape[2])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)

        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: image})
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.int64)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [12]:
class Ui_MainWindow(object):
    def setupUi(self, MainWindow):
        MainWindow.setObjectName("MainWindow")
        MainWindow.resize(800, 600)
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setObjectName("centralwidget")
        self.open = QtWidgets.QPushButton(self.centralwidget)
        self.open.setGeometry(QtCore.QRect(40, 80, 75, 23))
        self.open.setObjectName("open")
        self.Select = QtWidgets.QComboBox(self.centralwidget)
        self.Select.setGeometry(QtCore.QRect(670, 60, 91, 22))
        self.Select.setObjectName("Select")
        self.Select.addItem("")
        self.Select.setItemText(0, "Select Model")
        self.Select.addItem("")
        self.Select.addItem("")
        self.Select.addItem("")
        self.Select.addItem("")        
        self.next = QtWidgets.QPushButton(self.centralwidget)
        self.next.setGeometry(QtCore.QRect(40, 210, 75, 23))
        self.next.setObjectName("next")
        self.prev = QtWidgets.QPushButton(self.centralwidget)
        self.prev.setGeometry(QtCore.QRect(40, 350, 71, 21))
        self.prev.setObjectName("prev")
        self.save = QtWidgets.QPushButton(self.centralwidget)
        self.save.setGeometry(QtCore.QRect(40, 480, 75, 23))
        self.save.setObjectName("save")
        self.label1 = QtWidgets.QLabel(self.centralwidget)
        self.label1.setGeometry(QtCore.QRect(670, 210, 47, 13))
        self.label1.setObjectName("label1")
        self.label2 = QtWidgets.QLabel(self.centralwidget)
        self.label2.setGeometry(QtCore.QRect(670, 230, 47, 13))
        self.label2.setObjectName("label2")
        self.thres = QtWidgets.QLineEdit(self.centralwidget)
        self.thres.setGeometry(QtCore.QRect(740, 220, 31, 20))
        self.thres.setObjectName("thres")
        self.Filter = QtWidgets.QComboBox(self.centralwidget)
        self.Filter.setGeometry(QtCore.QRect(670, 300, 81, 21))
        self.Filter.setObjectName("Filter")
        self.Filter.addItem("")
        self.Filter.setItemText(0, "Label Filter")
        self.Filter.addItem("")
        self.Filter.addItem("")
        self.Filter.addItem("")
        self.Filter.addItem("")
        self.Filter.addItem("")
        self.Detect = QtWidgets.QPushButton(self.centralwidget)
        self.Detect.setGeometry(QtCore.QRect(660, 480, 111, 51))
        self.Detect.setObjectName("Detect")
        self.photo = QtWidgets.QLabel(self.centralwidget)
        self.photo.setGeometry(QtCore.QRect(160, 50, 471, 461))
        self.photo.setText("")
        self.photo.setPixmap(QtGui.QPixmap(str(TEST_IMAGE_PATHS[0])))
        self.photo.setScaledContents(True)
        self.photo.setWordWrap(False)
        self.photo.setObjectName("photo")
        self.label = QtWidgets.QLabel(self.centralwidget)
        self.label.setGeometry(QtCore.QRect(30, 110, 101, 20))
        self.label.setObjectName("label")
        self.file_loc = QtWidgets.QLineEdit(self.centralwidget)
        self.file_loc.setGeometry(QtCore.QRect(20, 140, 113, 20))
        self.file_loc.setObjectName("file_loc")
        self.get_thresh = QtWidgets.QPushButton(self.centralwidget)
        self.get_thresh.setGeometry(QtCore.QRect(670, 250, 75, 23))
        self.get_thresh.setObjectName("get_thresh")
        self.get_model = QtWidgets.QPushButton(self.centralwidget)
        self.get_model.setGeometry(QtCore.QRect(670, 100, 75, 23))
        self.get_model.setObjectName("get_model")
        self.get_filter = QtWidgets.QPushButton(self.centralwidget)
        self.get_filter.setGeometry(QtCore.QRect(670, 340, 75, 23))
        self.get_filter.setObjectName("get_filter")
        MainWindow.setCentralWidget(self.centralwidget)
        self.menubar = QtWidgets.QMenuBar(MainWindow)
        self.menubar.setGeometry(QtCore.QRect(0, 0, 800, 21))
        self.menubar.setObjectName("menubar")
        self.menuFile = QtWidgets.QMenu(self.menubar)
        self.menuFile.setObjectName("menuFile")
        MainWindow.setMenuBar(self.menubar)
        self.statusbar = QtWidgets.QStatusBar(MainWindow)
        self.statusbar.setObjectName("statusbar")
        MainWindow.setStatusBar(self.statusbar)
        self.actionNew = QtWidgets.QAction(MainWindow)
        self.actionNew.setObjectName("actionNew")
        self.actionSave = QtWidgets.QAction(MainWindow)
        self.actionSave.setObjectName("actionSave")
        self.actionOpen = QtWidgets.QAction(MainWindow)
        self.actionOpen.setObjectName("actionOpen")
        self.menuFile.addAction(self.actionSave)
        self.menuFile.addAction(self.actionOpen)
        self.menubar.addAction(self.menuFile.menuAction())

        self.retranslateUi(MainWindow)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)
        
        self.next.clicked.connect(self.show_next)
        self.prev.clicked.connect(self.show_prev)
        self.open.clicked.connect(self.open_file)
        self.get_thresh.clicked.connect(self.get_threshold)
        self.get_model.clicked.connect(self.select_model)
        self.get_filter.clicked.connect(self.select_filter)
        self.Detect.clicked.connect(self.detect_clicked)
        self.save.clicked.connect(self.save_file)
        

    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "MainWindow"))
        self.open.setText(_translate("MainWindow", "Open Folder"))
        self.Select.setItemText(1, _translate("MainWindow", "faster_rcnn_inception_v2_coco_2018_01_28"))
        self.Select.setItemText(2, _translate("MainWindow", "ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03"))
        self.Select.setItemText(3, _translate("MainWindow", "ssd_mobilenet_v1_coco_2018_01_28"))
        self.Select.setItemText(4, _translate("MainWindow", "ssd_inception_v2_coco_2017_11_17"))
        self.next.setText(_translate("MainWindow", "Next"))
        self.prev.setText(_translate("MainWindow", "Previous"))
        self.save.setText(_translate("MainWindow", "Save"))
        self.label1.setText(_translate("MainWindow", "Detection"))
        self.label2.setText(_translate("MainWindow", "Threshold"))
        self.Filter.setItemText(1, _translate("MainWindow", "Person"))
        self.Filter.setItemText(2, _translate("MainWindow", "Cat"))
        self.Filter.setItemText(3, _translate("MainWindow", "Dog"))
        self.Filter.setItemText(4, _translate("MainWindow", "Bottle"))
        self.Filter.setItemText(5, _translate("MainWindow", "Chair"))
        self.Detect.setText(_translate("MainWindow", "Detect"))
        self.label.setText(_translate("MainWindow", "Iput Folder Location:"))
        self.get_thresh.setText(_translate("MainWindow", "Enter"))
        self.get_model.setText(_translate("MainWindow", "Enter"))
        self.get_filter.setText(_translate("MainWindow", "Enter"))
        self.menuFile.setTitle(_translate("MainWindow", "File"))
        self.actionNew.setText(_translate("MainWindow", "New"))
        self.actionNew.setShortcut(_translate("MainWindow", "Ctrl+N"))
        self.actionSave.setText(_translate("MainWindow", "Save"))
        self.actionSave.setShortcut(_translate("MainWindow", "Ctrl+S"))
        self.actionOpen.setText(_translate("MainWindow", "Open"))
        self.actionOpen.setShortcut(_translate("MainWindow", "Ctrl+O"))
        
    def show_next(self):
        global i
        i = i+1
        print(i)
        self.photo.setPixmap(QtGui.QPixmap(str(TEST_IMAGE_PATHS[i])))

    def show_prev(self):
        global i
        i = i-1
        print(i)
        self.photo.setPixmap(QtGui.QPixmap(str(TEST_IMAGE_PATHS[i])))
    
    def open_file(self):
        global i
        i = 0
        path = self.file_loc.text()
        print(path)
        global PATH_TO_TEST_IMAGES_DIR, TEST_IMAGE_PATHS
        PATH_TO_TEST_IMAGES_DIR = pathlib.Path(path)
        TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
        TEST_IMAGE_PATHS
       
    def get_threshold(self):
        global threshold
        threshold = self.thres.text()
        print(threshold)
        
    def detect_clicked(self):
        global threshold, co_ordinate, items
        image = Image.open(TEST_IMAGE_PATHS[i])
        image_np = load_image_into_numpy_array(image) 
        image_np_expanded = np.expand_dims(image_np, axis=0)
        output_dict = run_inference_for_single_image(image_np_expanded, detection_graph)
        vis_util.visualize_boxes_and_labels_on_image_array(
            image_np,
            output_dict['detection_boxes'],
            output_dict['detection_classes'],
            output_dict['detection_scores'],
            category_index,
            instance_masks=output_dict.get('detection_masks'),
            use_normalized_coordinates=True,
            line_thickness=8)
#  display(Image.fromarray(image_np))
        image_mod = Image.fromarray(image_np)
        image_mod.save(image_save_path+str(i)+'.jpg')
#  print(image_path)
        self.photo.setPixmap(QtGui.QPixmap(image_save_path+str(i)+'.jpg'))
        get_inference.get_thresh(threshold)
        co_ordinate, items = get_inference.get_roi_label(np.array(Image.open(image_save_path+str(i)+'.jpg')))
    
    def select_model(self):
        global MODEL_NAME,detection_graph
        MODEL_NAME = self.Select.currentText()
        print(MODEL_NAME)
        MODEL_FILE = MODEL_NAME + '.tar.gz'
        PATH_TO_FROZEN_GRAPH ='models/research/object_detection/test_ckpt/'+MODEL_NAME+'/frozen_inference_graph.pb'
        detection_graph = tf.Graph()
        with detection_graph.as_default():
            od_graph_def = tf.compat.v1.GraphDef()
            with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
                serialized_graph = fid.read()
                od_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(od_graph_def, name='')

            sess = tf.compat.v1.Session(graph=detection_graph)
        
    def select_filter(self):
        global FILTER
        FILTER = self.Filter.currentText()
        print(FILTER)
        
    def save_file(self):
        global co_ordinate, items
        root=Element('box')
        tree=ElementTree(root)
        coordinates=Element('coordinates')
        root.append(coordinates)
        coordinates.text=str(co_ordinate)
        label=Element('label')
        root.append(label)
        label.text=str(items)
        print(etree.tostring(root))
        tree.write('models/research/object_detection/annoted_images/mod_image'+str(i)+'.xml', encoding='UTF-8', xml_declaration = True)

In [ ]:
if __name__ == "__main__":
    import sys
    app = QtWidgets.QApplication(sys.argv)
    MainWindow = QtWidgets.QMainWindow()
    ui = Ui_MainWindow()
    ui.setupUi(MainWindow)
    MainWindow.show()
    sys.exit(app.exec_())

file:///C:/Users/umang/Anaconda3/envs/tensorflow/intern_project


OSError: [WinError 123] The filename, directory name, or volume label syntax is incorrect: 'file:\\C:\\Users\\umang\\Anaconda3\\envs\\tensorflow\\intern_project'

models/research/object_detection/test_images
1
2
3
2
1
2
faster_rcnn_inception_v2_coco_2018_01_28
0.7
3
4
5
4
3
2
1
